# HW2 - Movies!




**Submission Instruction**

1- Replace the blank in the notebook title with your name (your notebook name should read like DE_HW2_Sara_Riazi)

2- Complete the notebook

3- Run all cells in notebook (all cells should have output).

4- Download .ipynb &nbsp;

5- Submit the .ipynb on the Gradescope

6- Double check your submission to make sure all cells have output

## Wranging and aggregating movie review data

The website MovieLens.com has a research group which provides open access to millions of reviews, for free!  We're going to work with those data for this homework.  [Feel free to check out the website here.](https://grouplens.org/datasets/movielens/)  You can go and download the raw data, but in order to make things a bit easier, the files have been uploaded the Google drive for fast direct downloads.  

There are two datasets we'll be working with.
* movies - this is a file of 60,000+ movies
* reviews - this is a file of 25 *million* individual reviews for the 60k movies

The goal for this section of the homework is to do two types of data aggregations that will allow for someone to make inferences on which movies were the most popular, reviewed, polarizing and were cult classics.  

To do this we'll first start by making a simple data set that brings just overall review properties together with movies.  We'll then do some deeper groupings to create a dataset that looks at the same properties but over time.

### Data first

Let's bring in our two files and libraries.  The ratings file is understandably large.  So it's a good idea to download it and then save a copy as something else and work with that.  This way if you mess up you don't have to download it all over again


In [45]:
# Libraries
import pandas as pd
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np



In [46]:
# Movies data
movies = pd.read_csv('http://131.193.32.85:9000/mybucket/movies.csv')
movies_backup = movies

In [47]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
62418,209157,We (2018),Drama
62419,209159,Window of the Soul (2001),Documentary
62420,209163,Bad Poems (2018),Comedy|Drama
62421,209169,A Girl Thing (2001),(no genres listed)


In [48]:
# Ratings data
ratings = pd.read_csv('http://131.193.32.85:9000/mybucket/ratings.csv')

In [49]:
# Make a copy of ratings; Generally, it's good practice to keep original copies saved before starting to mess with them
# Also if you want to re-do some of the cells you can use rating_backup instead of loading data from servers!
ratings_backup = ratings.copy

## Explore your data - [5 points]

Below take some time to explore your date.  
In one cell, check the following items for **both** datasets:

* Head and tail
* Shape
* Datatypes

In a new cell:
* The number of NaN values for the rating column of the ratings dataset

**Task** Do the head,tail, and shape operations all in one cell.  Count the number of NaNs in another.

In [50]:
# shape of movies
print("movies shape:", movies.shape)


movies shape: (62423, 3)


In [51]:
#head of movies
print("movies head:", movies.head())

movies head:    movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


In [52]:
#tail of movies
print("movies Tail:", movies.tail())

movies Tail:        movieId                           title                  genres
62418   209157                       We (2018)                   Drama
62419   209159       Window of the Soul (2001)             Documentary
62420   209163                Bad Poems (2018)            Comedy|Drama
62421   209169             A Girl Thing (2001)      (no genres listed)
62422   209171  Women of Devil's Island (1962)  Action|Adventure|Drama


In [53]:
# Get count of NaNs for movies
print("Number of NaNs in movies: ", movies.isnull().sum())

Number of NaNs in movies:  movieId    0
title      0
genres     0
dtype: int64


In [54]:
#Do the same steps for shape, head, and tail, and NaNs of ratings
print("ratings Shape:", ratings.shape)
print("ratings Head:", ratings.head())
print("ratings Tail:", ratings.tail())
print("number of NaNs in ratings:", ratings.isnull().sum())


ratings Shape: (25000095, 4)
ratings Head:    userId  movieId  rating   timestamp
0       1      296     5.0  1147880044
1       1      306     3.5  1147868817
2       1      307     5.0  1147868828
3       1      665     5.0  1147878820
4       1      899     3.5  1147868510
ratings Tail:           userId  movieId  rating   timestamp
25000090  162541    50872     4.5  1240953372
25000091  162541    55768     2.5  1240951998
25000092  162541    56176     2.0  1240950697
25000093  162541    58559     4.0  1240953434
25000094  162541    63876     5.0  1240952515
number of NaNs in ratings: userId       0
movieId      0
rating       0
timestamp    0
dtype: int64


## Convert timestamp in ratings to a datetime. - [2 points]

One issue that you can see from your exploration is that the ratings only have a timestamp.  This timestamp is measured in the number of seconds since 00:00:00 on January 1st, 1970.  You'll need to convert this to a datetime in order to actually do our later data aggregations.  

You use `pd.to_datetime` on timestamps like this.  [For full details on the various ways to use this function please look at the Pandas documentation.](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html) Briefly, this is how it's used.

```
>>> pd.to_datetime(1490195805, unit='s')
Timestamp('2017-03-22 15:16:45')
```

**Task** Make a new column in the movies dataframe called `review_dt` that contains the data from the timestamp column but converted to a datetime datatype.  

Note, it's good practice to do this by first assigning the output to a test vector first, rather than directly adding the output to the dataframe. This allows you to make sure your operation did what you wanted before modifying your dataframe. Once you're confident of the output, you can assign the vector to the `ratings` dataframe as a new column.

For example
```
test_vec = pd.to_datetime(arguments)
test_vec # to check what it contains
```

In [55]:
# Make test vector
print("Converting timestamp to datetime...")
review_dt = pd.to_datetime(ratings["timestamp"], unit='s') 

Converting timestamp to datetime...


In [56]:
# Check vector and datatype to make sure it make sense.
print(f"Head of review_dt vector:\n{review_dt.head()}\nDatatype of vector:]\n{review_dt.dtypes}")

#debugging print
print(f"Max of review_dt vector:\n{review_dt.max()}\nMin of review_dt vector:\n{review_dt.min()}")

Head of review_dt vector:
0   2006-05-17 15:34:04
1   2006-05-17 12:26:57
2   2006-05-17 12:27:08
3   2006-05-17 15:13:40
4   2006-05-17 12:21:50
Name: timestamp, dtype: datetime64[ns]
Datatype of vector:]
datetime64[ns]
Max of review_dt vector:
2019-11-21 09:15:03
Min of review_dt vector:
1995-01-09 11:46:49


In [57]:
# Now add this to the ratings dataframe
ratings["review_dt"] = review_dt

In [58]:
# Check the head of your dataframe again
print("ratings Head:\n", ratings.head())
print("ratings datatypes:\n", ratings.dtypes)


ratings Head:
    userId  movieId  rating   timestamp           review_dt
0       1      296     5.0  1147880044 2006-05-17 15:34:04
1       1      306     3.5  1147868817 2006-05-17 12:26:57
2       1      307     5.0  1147868828 2006-05-17 12:27:08
3       1      665     5.0  1147878820 2006-05-17 15:13:40
4       1      899     3.5  1147868510 2006-05-17 12:21:50
ratings datatypes:
 userId                int64
movieId               int64
rating              float64
timestamp             int64
review_dt    datetime64[ns]
dtype: object


In [59]:
# What's the oldest and newest review?  It should be 1995-01-09 and 2019-11-21, respectively.
print(f"oldest review:\n{ratings["review_dt"].min()}")
print(f"newet review:\n{ratings["review_dt"].max()}")


oldest review:
1995-01-09 11:46:49
newet review:
2019-11-21 09:15:03
newet review:
2019-11-21 09:15:03


## Your first aggregation and join - [8 points]

The first aggregation and join I want you to do is at the whole movie level.  Your movies dataframe should only have a single row for each movie, but there are obviously thousands of individual reviews for each of those movies. Our goal here is to produce some summary statistics about the reviews for each movie and then join them to the movies dataframe.

**Task:** Do the following:
* Create an aggregated dataframe called `ratings_by_movie`. This dataframe should be grouped by movie. Use `.agg()` to calculate the mean, standard deviation ('std'), and the number of reviews for each movie.  
* Rename the columns of that dataframe `'movieId', 'rating_mean', 'rating_std', and 'rating_count'`
* Join this new `ratings_by_movie` dataframe such that it attaches all those summary statistics to their corresponding movies from the 'movies' dataframe.
* Call the joined dataframe `movies_with_ratings`

In [60]:
# Make ratings_by_movie
ratings_by_movie = ratings.groupby(by="movieId", as_index=False).agg(
    rating_mean = ('rating', 'mean'),
    rating_std = ('rating', 'std'),
    rating_count = ('rating', 'count')
)

In [61]:
# Check it
print(f'Head of intial ratings_by_movie\n{ratings_by_movie.head()}')

Head of intial ratings_by_movie
   movieId  rating_mean  rating_std  rating_count
0        1     3.893708    0.921552         57309
1        2     3.251527    0.959851         24228
2        3     3.142028    1.008443         11804
3        4     2.853547    1.108531          2523
4        5     3.058434    0.996611         11714


In [62]:
# Rename columns
ratings_by_movie.columns = ['movieId', 'rating_mean', 'rating_std', 'rating_count']
print(f'Head of ratings_by_movie after column rename\n{ratings_by_movie.head()}')

Head of ratings_by_movie after column rename
   movieId  rating_mean  rating_std  rating_count
0        1     3.893708    0.921552         57309
1        2     3.251527    0.959851         24228
2        3     3.142028    1.008443         11804
3        4     2.853547    1.108531          2523
4        5     3.058434    0.996611         11714


In [63]:
# Join it and call movies with ratings
movies_with_ratings = movies.merge(ratings_by_movie, on='movieId', how='left')

In [64]:
# Check movies_with_ratings.
print(f'Head of movies_with_ratings\n{movies_with_ratings.head()}')
print(f'Shape of movies_with_ratings\n{movies_with_ratings.shape}')


Head of movies_with_ratings
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  rating_mean  rating_std  \
0  Adventure|Animation|Children|Comedy|Fantasy     3.893708    0.921552   
1                   Adventure|Children|Fantasy     3.251527    0.959851   
2                               Comedy|Romance     3.142028    1.008443   
3                         Comedy|Drama|Romance     2.853547    1.108531   
4                                       Comedy     3.058434    0.996611   

   rating_count  
0       57309.0  
1       24228.0  
2       11804.0  
3        2523.0  
4       11714.0  
Shape of movies_with_ratings
(62423, 6)


Is your merged dataframe 62423 rows × 6 columns?

## Filtering and more transformations - [5 points]

Now we want to clean up this dataset a bit and then do a couple more transforms. One issue you can see from your check above is that many movies only have one rating.  We're going to choose to set a minimum number of reviews needed to be included.  We also want to do some binning where movies with certain ratings levels

**Task:** Please do the following operations
* Filter `movies_with_ratings` so it only contains movies that have at least 10 ratings
* Use the function `cut()` to automatically bin our `rating_mean` column into three groups of 'bad', 'fine', or 'good' movies.  Call this `rating_group`.
* Use the same function to take the standard deviation in rating and make three groups of 'agreement', 'average', 'controversial'.  Thus, movies with low standard deviation have agreement in the rating, while movies with high standard deviation have controversy in the ratings.  Call this column `ratings_agreement`.

In [65]:
# Filter first being sure to overwrite dataframe
movies_with_ratings = movies_with_ratings[movies_with_ratings['rating_count'] >= 10]

In [66]:
# Check how many rows you're left with. You should have a little over 24000
print(f'Shape of filtered movies_with_ratings\n{movies_with_ratings.shape}')

Shape of filtered movies_with_ratings
(24330, 6)


I didn't show you how to use `cut()` in the lesson, but it's a transform just like anything else.  You could use a `np.where()` statement like we did, but `cut()` is a bit easier.  All it does is take 1) a column as the first argument, 2) the number of bins you want to group it in as the second argument, and then 3) the labels you want to give those bins as the third.  It automatically divides them up into equal sized bins.

For example, if I make the following list:
```
rating = [2, 4, 9, 8, 5, 3, 6, 10, 2, 1, 6, 7]
```

And run `cut()` on it with three bins and levels 'bad', 'fine', and 'good':
```
pd.cut(rating, 3, labels=['bad', 'fine', 'good'])
```

I get a return of:
```
[bad, bad, good, good, fine, ..., good, bad, bad, fine, fine]
Length: 12
Categories (3, object): [bad < fine < good]
```

Note how it orders them for you based on the order of the labels.  

In [67]:
# If you want to test it!
rating = [2, 4, 9, 8, 5, 3, 6, 10, 2, 1, 6, 7]
pd.cut(rating, 3, labels=['bad', 'fine', 'good'])

['bad', 'bad', 'good', 'good', 'fine', ..., 'good', 'bad', 'bad', 'fine', 'fine']
Length: 12
Categories (3, object): ['bad' < 'fine' < 'good']

Note: if get a SettingWithCopyWarning that's ok here.
Pandas tries to warn us that we are working with a copy of movies_with_rating,
and the changes won't reflace back to the original movies_with_rating dataframe. But that's ok for us

In [68]:
# Now make 'bad', 'fine', 'good' levels for ratings
# Assign to new column called 'rating_group'
movies_with_ratings['rating_group'] = pd.cut(movies_with_ratings['rating_mean'], 3, labels=['bad', 'fine', 'good'])

In [69]:
# Check it
movies_with_ratings

,movieId,title,genres,rating_mean,rating_std,rating_count,rating_group
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.893708,0.921552,57309.0,good
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.251527,0.959851,24228.0,good
2,3,Grumpier Old Men (1995),Comedy|Romance,3.142028,1.008443,11804.0,fine
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.853547,1.108531,2523.0,fine
4,5,Father of the Bride Part II (1995),Comedy,3.058434,0.996611,11714.0,fine
...,...,...,...,...,...,...,...
61967,207405,Doctor Sleep (2019),Horror,3.730769,0.785109,39.0,good
62027,207612,Tell Me Who I Am (2019),Documentary,3.357143,1.099450,14.0,good
62088,207830,Terminator: Dark Fate (2019),Action|Sci-Fi,3.372727,0.794764,55.0,good
62313,208715,Let It Snow (2019),Comedy|Romance,2.933333,1.222799,15.0,fine


Do Toy Story and Jumanji have 'good' ratings?  Does Grumpier Old Men have a 'fair' rating?

In [70]:
# Now use cut() again to create your ratings_agreement column.
# Use three bins and order of 'agreement', 'average',  and 'controversial'
movies_with_ratings['ratings_agreement'] = pd.cut(movies_with_ratings['rating_std'], 3, labels=['agreement', 'average', 'controversial'])

In [71]:
# Check to make sure that your bin categories make sense
print("\nRating Group Statistics:")
print(movies_with_ratings.groupby('rating_group')['rating_mean'].describe())

print("\nRating Agreement Statistics:")
print(movies_with_ratings.groupby('ratings_agreement')['rating_std'].describe())


Rating Group Statistics:
                count      mean       std       min       25%       50%  \
rating_group                                                              
bad             426.0  1.688237  0.228236  0.678571  1.597436  1.750000   
fine          10753.0  2.818177  0.304257  1.946970  2.630435  2.884968   
good          13151.0  3.562540  0.223832  3.215035  3.377778  3.538462   

                   75%       max  
rating_group                      
bad           1.854009  1.943946  
fine          3.071412  3.214552  
good          3.719940  4.483096  

Rating Agreement Statistics:
                     count      mean       std       min       25%       50%  \
ratings_agreement                                                              
agreement           1132.0  0.571619  0.118814  0.000000  0.526502  0.607870   
average            22244.0  1.001487  0.148103  0.690497  0.892427  0.989153   
controversial        954.0  1.502402  0.106286  1.381137  1.420151  1.474

C:\Users\couga\AppData\Local\Temp\ipykernel_36576\2851411417.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(movies_with_ratings.groupby('rating_group')['rating_mean'].describe())
C:\Users\couga\AppData\Local\Temp\ipykernel_36576\2851411417.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(movies_with_ratings.groupby('ratings_agreement')['rating_std'].describe())


###  Exploring our data [2 points]

Making bins like this allows us to figure out things like which movies are both bad, but have differing opinions on.  For example, we can filter out movies that are in the bad category but have a lot of controversy about those ratings.  This could mean they're 'cult classic' movies where despite the low rating some people actually really love the movies.  

A dataset like this could be used in a recommendation engine where if you see people liking these 'bad but good' movies you could suggest others that meet the came criteria.  

**Task:** You don't have to do anything, the code to let you figure out if you have done everything correctly so far.

In [72]:
movies_with_ratings[(movies_with_ratings['rating_group'] == 'bad') &
                    (movies_with_ratings['ratings_agreement'] == 'controversial') &
                    (movies_with_ratings['rating_count'] >= 100)]

,movieId,title,genres,rating_mean,rating_std,rating_count,rating_group,ratings_agreement
8618,26157,Manos: The Hands of Fate (1966),Horror,1.798165,1.550268,109.0,bad,controversial
16964,89281,Birdemic: Shock and Terror (2010),Romance|Thriller,1.521739,1.503492,115.0,bad,controversial


If you have done everything correctly so far, you will get:
The Hands of Fate (1966)	Horror and	Birdemic: Shock and Terror (2010)	as the outcome.


## Grouping within years - [5 points]

Now that we've done our overall grouping by movie, let's get a bit more detail about these ratings. Specifically, let's engineer a dataset that breaks down the average rating not only by movie, but also by the year the person provided the review.  This would allow for someone to see which movies continue to do well over time, which ones become more popular, and which ones don't age well!

**Task:** You're going to do the following steps:
* Create a new `ratings_by_movie_year` that groups both by `movieId` but also by your `review_dt` column.  I want you to group into year intervals.  
* Join `movies` to `ratings_by_movie_year` so that you have the summary review statistics for each year the movie has been out
* Clean up and filter your dataframe

First, create `ratings_by_movie`.  You can group by two levels by just adding a list of what levels you want to group by in the `groupby()` statement.  I'll give you some help there, but you have to complete the rest in order to group by movieId first and review_dt second.

In [73]:
# Note the groupby syntax.  I first am grouping by movieId
# But then also am calling dt.year on our datetime column
# This will then tell Python to do the aggregations within year as well
ratings_by_movie_year = ratings.groupby(['movieId', ratings['review_dt'].dt.year]).agg({
    'rating': ['mean', 'std', 'count']
})

In [74]:
# Check
...

You need to rename columns, but this is a bit trickier as you have two levels of your dataframe index. I'm going to give you the code below.  But, what it's doing is resetting that one level of the index `review_dt` and putting it back as a regular column.  I'm then renaming the resulting columns.

In [75]:
# Reset index (you cannot rerun this cell (or any cell that includes reset_index) twice unless you re-create ratings_by_movie_year)
ratings_by_movie_year = ratings_by_movie_year.reset_index(level = 'review_dt')
ratings_by_movie_year.columns = ['year', 'rating_mean', 'rating_std', 'rating_count']
ratings_by_movie_year

,year,rating_mean,rating_std,rating_count
movieId,,,,
1,1996,4.132756,0.884990,6237
1,1997,3.872500,0.894079,6000
1,1998,3.889515,0.944972,887
1,1999,3.975140,0.913960,2494
1,2000,4.136634,0.865127,3535
...,...,...,...,...
209157,2019,1.500000,NaN,1
209159,2019,3.000000,NaN,1
209163,2019,4.500000,NaN,1


In [76]:
# Now join the movie dataframe onto ratings_by_movie_year
ratings_by_movie_year = movies.merge(ratings_by_movie_year, on='movieId', how='inner')

In [77]:
# Check the resulting dataframe
print("First few rows:")
print(ratings_by_movie_year.head())
print("\nDataframe Info:")
print(ratings_by_movie_year.info())

First few rows:
   movieId             title                                       genres  \
0        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
1        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
2        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
3        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
4        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   

   year  rating_mean  rating_std  rating_count  
0  1996     4.132756    0.884990          6237  
1  1997     3.872500    0.894079          6000  
2  1998     3.889515    0.944972           887  
3  1999     3.975140    0.913960          2494  
4  2000     4.136634    0.865127          3535  

Dataframe Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 323737 entries, 0 to 323736
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   movieId

In [78]:
# How many rows are there in the resulting dataframe?
print(f"Number of rows: {len(ratings_by_movie_year)}")
print(f"Shape of dataframe: {ratings_by_movie_year.shape}")

# Check if Toy Story's 1996 rating is correct
toy_story_1996 = ratings_by_movie_year[
    (ratings_by_movie_year['title'].str.contains('Toy Story', na=False)) & 
    (ratings_by_movie_year['year'] == 1996)
]
print("\nToy Story's 1996 ratings:")
print(toy_story_1996)

Number of rows: 323737
Shape of dataframe: (323737, 7)

Toy Story's 1996 ratings:
   movieId             title                                       genres  \
0        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   

   year  rating_mean  rating_std  rating_count  
0  1996     4.132756     0.88499          6237  


In your dataframe do you see Toy Story having a mean rating of 4.132756 in 1996?  Is your dataframe 323737 rows × 7 columns?

### A quick plot  [3 points]

Now you have a dataset where one could explore how movies have done over time. I've made a couple plots below to show you want I mean. You don't have to add new code here. As long as you can generate a plausible plot, it means that you have done everything so far correctly and you get the points

In [79]:
# What movie has the max rating?
movies_with_ratings.loc[movies_with_ratings['rating_count'] == movies_with_ratings['rating_count'].max()]

,movieId,title,genres,rating_mean,rating_std,rating_count,rating_group,ratings_agreement
351,356,Forrest Gump (1994),Comedy|Drama|Romance|War,4.048011,0.93881,81491.0,good,average



If you have done everything correctly you should get Forrest Gump (1994)

In [80]:
# Grabbing just the ID for that movie.
movies_with_ratings['movieId'][movies_with_ratings['rating_count'] == movies_with_ratings['rating_count'].max()].values[0]

np.int64(356)

In [81]:
# Forest gump has been reviewed over 80000 times!
# Let's extract that movie id to an object and then make just that dataframe
movies_with_ratings['movieId'][movies_with_ratings['rating_count'] == movies_with_ratings['rating_count'].max()].values[0]
most_viewed_df

NameError: name 'most_viewed_df' is not defined

In [ ]:
#  A quick lineplot shows that although forest gump has a really high rating on average, it seems to have some bad years for some reason.
sns.lineplot(data = most_viewed_df, x = 'year', y = 'rating_mean')

NameError: name 'most_viewed_df' is not defined

In [ ]:
# Let's look at the summary statistics for Forrest Gump across years
print("Summary statistics for Forrest Gump ratings by year:")
print(most_viewed_df.describe())

# Find the year with highest and lowest ratings
best_year = most_viewed_df.loc[most_viewed_df['rating_mean'].idxmax()]
worst_year = most_viewed_df.loc[most_viewed_df['rating_mean'].idxmin()]

print(f"\nBest rated year: {best_year['year']} with average rating {best_year['rating_mean']:.2f}")
print(f"Worst rated year: {worst_year['year']} with average rating {worst_year['rating_mean']:.2f}")

# Add a trend line to see the overall pattern
plt.figure(figsize=(10, 6))
sns.regplot(data=most_viewed_df, x='year', y='rating_mean', scatter_kws={'alpha':0.5})
plt.title("Forrest Gump Ratings Over Time with Trend")
plt.show()

You can also look up some of your favorite movies.  Here we are looking up 'Dredd'.  We can search in the titles for movies we like, and then call that ID to filter a new dataframe.  We can then make some plots

In [ ]:
# Search for Dredd
movies_with_ratings[movies_with_ratings['title'].str.contains('Dredd')]

In [ ]:
# Call the dredd to make a dataframe
dredd_id = movies_with_ratings['movieId'][movies_with_ratings['title'].str.contains('^Dredd')].values[0]
dredd_df = ratings_by_movie_year[ratings_by_movie_year['movieId'] == dredd_id]
dredd_df

In [ ]:
# Our plot shows that the year after release it was reviewed pretty poorly,
# but the score gradually grew as people realized how awesome it was :)
sns.lineplot(x = 'year', y = 'rating_mean', data = dredd_df)

In [ ]:
# Compare the rating trajectories
plt.figure(figsize=(12, 6))
sns.lineplot(data=most_viewed_df, x='year', y='rating_mean', label='Forrest Gump')
sns.lineplot(data=dredd_df, x='year', y='rating_mean', label='Dredd')
plt.title("Rating Comparison: Forrest Gump vs Dredd")
plt.legend()
plt.show()

# Print some statistics about both movies
print("\nForrest Gump Statistics:")
print(most_viewed_df['rating_mean'].describe())
print("\nDredd Statistics:")
print(dredd_df['rating_mean'].describe())